In [14]:
import yfinance
import numpy as np
import pandas as pd
import pandas
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import cufflinks as cf
cf.set_config_file(offline=True)

import warnings
warnings.filterwarnings("ignore")


In [15]:
from ActMng import AccountManager

In [16]:
starting_capital = 100_000
allocation = 0.8

symbol = 'SPY'

long_trend_days = 12
short_trend_days = 6

In [17]:
def fetch_data(symbol:str='SPY', start_date:str='2020', end_date:str=None):
    
    if end_date is None:
        end = pandas.Timestamp.now()
    else:
        end = end_date
    
    return pdr.get_data_yahoo(symbol, pandas.Timestamp(start_date), pandas.Timestamp(end))[['Open','Close']]
df = fetch_data(symbol)

[*********************100%***********************]  1 of 1 completed


In [18]:
df

,Open,Close
Date,,
2019-12-31,320.529999,321.859985
2020-01-02,323.540009,324.869995
2020-01-03,321.160004,322.410004
2020-01-06,320.489990,323.640015
2020-01-07,323.019989,322.730011
...,...,...
2021-11-04,465.359985,466.910004
2021-11-05,469.279999,468.529999
2021-11-08,469.700012,468.929993


In [19]:
fast_trend = df.Open.rolling(window=short_trend_days).mean()
slow_trend = df.Open.rolling(window=long_trend_days).mean()

df['ma_fast'] = fast_trend
df['ma_slow'] = slow_trend
df[['Close','ma_fast','ma_slow']].iplot()

In [20]:
df

,Open,Close,ma_fast,ma_slow
Date,,,,
2019-12-31,320.529999,321.859985,NaN,NaN
2020-01-02,323.540009,324.869995,NaN,NaN
2020-01-03,321.160004,322.410004,NaN,NaN
2020-01-06,320.489990,323.640015,NaN,NaN
2020-01-07,323.019989,322.730011,NaN,NaN
...,...,...,...,...
2021-11-04,465.359985,466.910004,459.751658,456.872498
2021-11-05,469.279999,468.529999,462.054993,458.384997
2021-11-08,469.700012,468.929993,464.359996,459.879166


In [21]:
df['trade_direction'] = np.sign(fast_trend-slow_trend)
df

,Open,Close,ma_fast,ma_slow,trade_direction
Date,,,,,
2019-12-31,320.529999,321.859985,NaN,NaN,NaN
2020-01-02,323.540009,324.869995,NaN,NaN,NaN
2020-01-03,321.160004,322.410004,NaN,NaN,NaN
2020-01-06,320.489990,323.640015,NaN,NaN,NaN
2020-01-07,323.019989,322.730011,NaN,NaN,NaN
...,...,...,...,...,...
2021-11-04,465.359985,466.910004,459.751658,456.872498,1.0
2021-11-05,469.279999,468.529999,462.054993,458.384997,1.0
2021-11-08,469.700012,468.929993,464.359996,459.879166,1.0


In [22]:
#Determine the direction of the moving average differences
df_trades = df['trade_direction'].dropna().diff().dropna().apply(np.sign).astype('int8')
df_trades = pandas.DataFrame(df_trades[df_trades !=0])

#Label our intended trades
df_trades['trade'] = df['trade_direction'].map({1:'Long',-1:'Short'})
df['trades'] = df_trades['trade']


In [23]:
df

,Open,Close,ma_fast,ma_slow,trade_direction,trades
Date,,,,,,
2019-12-31,320.529999,321.859985,NaN,NaN,NaN,NaN
2020-01-02,323.540009,324.869995,NaN,NaN,NaN,NaN
2020-01-03,321.160004,322.410004,NaN,NaN,NaN,NaN
2020-01-06,320.489990,323.640015,NaN,NaN,NaN,NaN
2020-01-07,323.019989,322.730011,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-11-04,465.359985,466.910004,459.751658,456.872498,1.0,NaN
2021-11-05,469.279999,468.529999,462.054993,458.384997,1.0,NaN
2021-11-08,469.700012,468.929993,464.359996,459.879166,1.0,NaN


In [24]:
#Label our opening trade after removing our window period
df['trades'].replace(pandas.np.NaN,'None',inplace=True)
df = df.dropna()
df['trades'].iloc[0]=('Long' if df['trade_direction'].iloc[0]==1.0 else 'Short')
df.head(30)

,Open,Close,ma_fast,ma_slow,trade_direction,trades
Date,,,,,,
2020-01-16,329.700012,330.920013,327.393341,324.670003,1.0,Long
2020-01-17,331.700012,331.950012,328.316676,325.600838,1.0,None
2020-01-21,330.899994,331.299988,328.918340,326.214170,1.0,None
2020-01-22,332.239990,331.339996,329.893336,327.137502,1.0,None
2020-01-23,330.630005,331.720001,330.420003,327.982503,1.0,None
2020-01-24,332.440002,328.769989,331.268336,328.767504,1.0,None
2020-01-27,323.029999,323.500000,330.156667,328.775004,1.0,None
2020-01-28,325.059998,326.890015,329.049998,328.683337,1.0,None
2020-01-29,328.380005,326.619995,328.630000,328.774170,-1.0,Short


In [25]:
# Our trading strategy implementation - feel free to brose ActMng.py if you are curious.
trading_strategy = AccountManager(symbol=symbol,trades_df=df,starting_capital=starting_capital,allocation_amount=allocation)
results = trading_strategy.run()

In [26]:
results.keys()

dict_keys(['result_df', 'broker_orders'])

In [27]:
results_df = results["result_df"]
results_df.head()

,Open,Close,ma_fast,ma_slow,trade_direction,trades,account_val,cash_bal,position,position_changes,fees
2020-01-16,329.700012,330.920013,327.393341,324.670003,1.0,Long,100295.021096,20000.589038,242.64,242.64,1.0
2020-01-17,331.700012,331.950012,328.316676,325.600838,1.0,None,100544.940000,20000.589038,242.64,0.00,0.0
2020-01-21,330.899994,331.299988,328.918340,326.214170,1.0,None,100387.218076,20000.589038,242.64,0.00,0.0
2020-01-22,332.239990,331.339996,329.893336,327.137502,1.0,None,100396.925750,20000.589038,242.64,0.00,0.0
2020-01-23,330.630005,331.720001,330.420003,327.982503,1.0,None,100489.130134,20000.589038,242.64,0.00,0.0


In [28]:
# Here are our simulated orders as we would submit to an API endpoint for algorthmic trading. 
results['broker_orders']

,symbol,date,time,order,order_type,price,size,exchange_fees
2020-01-16,SPY,2020-01-16 00:00:00,At Market Open,BUY,MARKET,329.700012,242.64,1.0
2020-01-29,SPY,2020-01-29 00:00:00,At Market Open,SELL,MARKET,328.380005,242.64,1.0
2020-01-29,SPY,2020-01-29 00:00:00,At Market Open,SELL,MARKET,328.380005,242.84,1.0
2020-02-07,SPY,2020-02-07 00:00:00,At Market Open,BUY,MARKET,332.820007,242.84,1.0
2020-02-07,SPY,2020-02-07 00:00:00,At Market Open,BUY,MARKET,332.820007,237.00,1.0
...,...,...,...,...,...,...,...,...
2021-09-14,SPY,2021-09-14 00:00:00,At Market Open,SELL,MARKET,448.119995,223.86,1.0
2021-09-14,SPY,2021-09-14 00:00:00,At Market Open,SELL,MARKET,448.119995,223.90,1.0
2021-10-13,SPY,2021-10-13 00:00:00,At Market Open,BUY,MARKET,434.709991,223.90,1.0
2021-10-13,SPY,2021-10-13 00:00:00,At Market Open,BUY,MARKET,434.709991,236.33,1.0


In [29]:
broker_orders_df = results["broker_orders"]
broker_orders_df.head()

,symbol,date,time,order,order_type,price,size,exchange_fees
2020-01-16,SPY,2020-01-16 00:00:00,At Market Open,BUY,MARKET,329.700012,242.64,1.0
2020-01-29,SPY,2020-01-29 00:00:00,At Market Open,SELL,MARKET,328.380005,242.64,1.0
2020-01-29,SPY,2020-01-29 00:00:00,At Market Open,SELL,MARKET,328.380005,242.84,1.0
2020-02-07,SPY,2020-02-07 00:00:00,At Market Open,BUY,MARKET,332.820007,242.84,1.0
2020-02-07,SPY,2020-02-07 00:00:00,At Market Open,BUY,MARKET,332.820007,237.00,1.0


In [30]:
#Todo:

# Calculate the Absolute Return of our trend-following trading strategy
absolute_return_tf = results_df["account_val"].iloc[-1] / starting_capital - 1
print(f"Absolute Return = {absolute_return_tf:.5f}")


# Calculate the CAGR or our trend-following trading strategy
CAGR_tf = (results_df["account_val"].iloc[-1] / starting_capital) ** (252/len(df)) - 1
print(f"CAGR = {CAGR_tf:.5f}")


# Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
information_ratio_tf = (results_df["account_val"].pct_change().mean() / results_df["account_val"].pct_change().std()) * (252) ** 0.5
print(f"IR = {information_ratio_tf:.5f}")


##Plot the equity curve (hint: use 'account_val')
results_df["account_val"].iplot()


Absolute Return = 0.35250
CAGR = 0.17989
IR = 0.99296


In [31]:
#Todo:
# Calculate the Absolute Return of a buy-and-hold strategy in the same period
absolute_return_bh = (results_df["Close"].iloc[-1] / results_df["Close"].iloc[0]) - 1
print(f"Absolute Return = {absolute_return_bh:.5f}")


# Calculate the CAGR or our trend-following trading strategy
CAGR = (results_df["Close"].iloc[-1] / results_df["Close"].iloc[0]) ** (252/len(df)) - 1
print(f"CAGR = {CAGR:.5f}")


# Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
information_ratio = (results_df["Close"].pct_change().mean() / results_df["Close"].pct_change().std()) * np.sqrt(252)
print(f"IR = {information_ratio:.5f}")


#Plot the equity curve (hint: use 'close' returns and your initial capital value to generate an equity curve)
# Todo
(results_df["Close"].pct_change().add(1).cumprod()  * starting_capital * allocation + 20_000).iplot()


Absolute Return = 0.40100
CAGR = 0.20288
IR = 0.83726


In [ ]:
#Todo - optional, below:
# Copy any of the code from above to new cells below, and see how changing the symbol, dates and trend parameters changes the performance
# Can you beat the reference implementation?

# Optional 


In [1]:
from ActMng import AccountManager

import datetime as dt
import numpy as np
import pandas as pd
from scipy import optimize

import cufflinks as cf
import yfinance as yf

import plotly.graph_objects as go 
import plotly.express as px

In [2]:
cf.set_config_file(offline=True)

In [3]:
def sma_strategy(asset, **kwargs):
    """
    SMA Simple Moving Average: 
    
    Based on Open, long and short averages we wolud
    by at end on the day(Close price)
    
    **kwargs
    
    short_ma: Short (Fast) Moving Average
    long_ma: Long (Slow) Moving Average
    
    - Short Average > Long Average: BUY
    - Short Average < Long Average: SELL
    
    """
    short_ma = kwargs.get("short_ma", 50)
    long_ma = kwargs.get("long_ma", 100)
    
    df = asset.copy()
    df["Long MA"] = asset["Open"].rolling(long_ma).mean()
    df["Short MA"] = asset["Open"].rolling(short_ma).mean()
    df["trade_direction"] = np.where(df["Short MA"] > df["Long MA"], 1, -1)
    df = df.dropna()
    
    df["trades"] = df["trade_direction"].diff().replace({-2: "Short", 2: "Long", 0: "None"})
    
    df["trades"].iloc[0] = "Long" if df["trade_direction"].iloc[0] == 1 else "None"
    
    return df

In [4]:
def bh_strategy(asset):
    """
    Simple Buy Hold
    """
    
    df = asset.copy()
    df["trades"] = "None"
    df["trades"].iloc[0] = "Long" 
        
    return df

In [5]:
def get_performance(results, starting_capital):
    """
    Absolute Return
    
    CAGR"Compund Annual Growth Rate"
    
    Information Ratio
    """
    # Using 365 to be more general
    n = (results.index[-1] - results.index[0]).days
    
    ending_capital = results["account_val"].iloc[-1]
    
    returns = results["account_val"].pct_change()
    std = returns.std()
    
    return {
        "Absolute Return":  ending_capital / starting_capital - 1,
        "CAGR": (ending_capital / starting_capital) ** (365/n) - 1,
        "IR": (returns.mean() / std) * ((365) ** 0.5)
    }

In [6]:
def get_data(ticker, start_date, end_date=None):
    if end_date == None:
        end_date = dt.datetime.now()
    df = yf.download(ticker, start_date, end_date)[["Open", "Close"]]
    return df

In [7]:
class BackTesting():
    
    def __init__(self,
                 ticker,
                 start_date,
                 end_date=None,
                 starting_capital=100_000,
                 allocation = 0.8):
        
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.starting_capital = starting_capital
        self.allocation = allocation
        self.asset = get_data(self.ticker, self.start_date, self.end_date)
        
    def run(self, strategy, **kwargs):
        trades = strategy(self.asset, **kwargs)
        test_strategy = AccountManager(symbol=self.ticker,
                                       trades_df=trades,
                                       starting_capital=self.starting_capital,
                                       allocation_amount=self.allocation
                                      )
        results, orders = test_strategy.run().values()
        performance = get_performance(results, self.starting_capital) 
        return performance, results

In [8]:
def plot_signal(asset, short_mv, long_mv):

    fig = go.Figure()

    sell_signal = asset.query("trades == 'Short'").index
    buy_signal = asset.query("trades == 'Long'").index
    
    # Close Price
    fig.add_trace(go.Scatter(x=asset.index,
                             y=asset["Close"],
                             name="Price",
                             ))
    # Short Average
    fig.add_trace(go.Scatter(x=asset.index,
                             y=asset["Short MA"],
                             name="SMA Short",
                             ))
    # Long Average
    fig.add_trace(go.Scatter(x=asset.index,
                             y=asset["Long MA"],
                             name="SMA Long",
                             ))

    # Sell(Short) Signal
    fig.add_trace(go.Scatter(x=sell_signal,
                             y=asset.loc[sell_signal, "Close"],
                             mode="markers",
                             name="Sell",
                             marker=dict(color="Red",
                                         symbol="triangle-down",
                                         size=15,
                                         line={"width": 1,
                                               "color": "DarkSlateGrey"},
                                         ),

                             ))
    # Buy(Long) Signal
    fig.add_trace(go.Scatter(x=buy_signal,
                             y=asset.loc[buy_signal, "Close"],
                             mode="markers",
                             name="Buy",
                             marker=dict(color="Green",
                                         symbol="triangle-up",
                                         size=15,
                                         line={"width": 1,
                                               "color": "DarkSlateGrey"},
                                         ),
                             ))

    fig.update_layout(dict(xaxis={"title": "Time",
                                  "rangeslider_visible": True,
                                  },
                           yaxis={"title": "Price"},
                           title=f"SMA Short {short_mv} | SMA Long {long_mv}",
                           legend={"orientation": "h",
                                   "yanchor": "bottom",
                                   "y": 1.02,
                                   "xanchor": "right",
                                   "x": 1},
                           height=1000
                      ))

    fig.show()

In [9]:
def plot_returns(benchmarck, strategy, name=None):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=benchmarck.index,
                             y=benchmarck["account_val"],
                             fill="tozeroy",
                             name="Buy & Hold Strategy"
                             ))

    fig.add_trace(go.Scatter(x=strategy.index,
                             y=strategy["account_val"],
                             fill="tozeroy",
                             name=name
                             ))

    fig.update_layout(dict(xaxis={"title": "Time",
                                  "rangeslider_visible": True,
                                  },
                           yaxis={"title": "$"},
                           title=name,
                           legend={"orientation": "h",
                                   "yanchor": "bottom",
                                   "y": 1.02,
                                   "xanchor": "right",
                                   "x": 1},
                           height=700,
                      ))
    fig.show()

In [10]:
short_ma = 6
long_ma = 12
test_spy = BackTesting("SPY", start_date=dt.datetime(2020, 1, 1))
sma_metrics, sma_results = test_spy.run(sma_strategy, short_ma=short_ma, long_ma=long_ma)
sma_metrics

[*********************100%***********************]  1 of 1 completed


{'Absolute Return': 0.35250167946472133,
 'CAGR': 0.1805551982589244,
 'IR': 1.1950287957176964}

In [11]:
sma_results.head(10)

,Open,Close,Long MA,Short MA,trade_direction,trades,account_val,cash_bal,position,position_changes,fees
2020-01-16,329.700012,330.920013,324.670003,327.393341,1,Long,100295.021096,20000.589038,242.64,242.64,1.0
2020-01-17,331.700012,331.950012,325.600838,328.316676,1,None,100544.940000,20000.589038,242.64,0.00,0.0
2020-01-21,330.899994,331.299988,326.214170,328.918340,1,None,100387.218076,20000.589038,242.64,0.00,0.0
2020-01-22,332.239990,331.339996,327.137502,329.893336,1,None,100396.925750,20000.589038,242.64,0.00,0.0
2020-01-23,330.630005,331.720001,327.982503,330.420003,1,None,100489.130134,20000.589038,242.64,0.00,0.0
2020-01-24,332.440002,328.769989,328.767504,331.268336,1,None,99773.339172,20000.589038,242.64,0.00,0.0
2020-01-27,323.029999,323.500000,328.775004,330.156667,1,None,98494.629038,20000.589038,242.64,0.00,0.0
2020-01-28,325.059998,326.890015,328.683337,329.049998,1,None,99317.182192,20000.589038,242.64,0.00,0.0
2020-01-29,328.380005,326.619995,328.774170,328.630000,-1,Short,100104.114194,179420.513809,-242.84,-485.48,2.0
2020-01-30,324.359985,327.679993,328.605001,327.316666,-1,None,99846.704387,179420.513809,-242.84,0.00,0.0


In [12]:
plot_signal(sma_results, short_ma, long_ma)

In [13]:
test_spy = BackTesting("SPY", start_date=dt.datetime(2020, 1, 16))
bh_metrics, bh_results = test_spy.run(bh_strategy)
bh_metrics

[*********************100%***********************]  1 of 1 completed


{'Absolute Return': 0.33302022615051285,
 'CAGR': 0.1708989409307038,
 'IR': 1.0445098832900306}

In [14]:
bh_results.head(10)

,Open,Close,trades,account_val,cash_bal,position,position_changes,fees
2020-01-15,327.350006,328.190002,Long,100204.286705,19997.932008,244.39,244.39,1.0
2020-01-16,329.700012,330.920013,None,100871.474090,19997.932008,244.39,0.00,0.0
2020-01-17,331.700012,331.950012,None,101123.195492,19997.932008,244.39,0.00,0.0
2020-01-21,330.899994,331.299988,None,100964.336025,19997.932008,244.39,0.00,0.0
2020-01-22,332.239990,331.339996,None,100974.113713,19997.932008,244.39,0.00,0.0
2020-01-23,330.630005,331.720001,None,101066.983107,19997.932008,244.39,0.00,0.0
2020-01-24,332.440002,328.769989,None,100346.029623,19997.932008,244.39,0.00,0.0
2020-01-27,323.029999,323.500000,None,99058.097008,19997.932008,244.39,0.00,0.0
2020-01-28,325.059998,326.890015,None,99886.582688,19997.932008,244.39,0.00,0.0
2020-01-29,328.380005,326.619995,None,99820.592615,19997.932008,244.39,0.00,0.0


In [15]:
plot_returns(bh_results, sma_results, name=f"SMA Strategy {short_ma} | {long_ma}")

In [ ]:
short_mas = (5, 50, 1)
long_mas = (100, 252, 1)
f = lambda x: -test_spy.run(sma_strategy, short_max=int(x[0]), long_ma=int(x[1]))[0]["CAGR"]
best_values = optimize.brute(f, (short_mas, long_mas))

In [ ]:
best_values

array([  5., 201.])

In [ ]:
best_metrics, best_result = test_spy.run(sma_strategy, short_ma=5, long_ma=201)
best_metrics

{'Absolute Return': 0.3345466910888675,
 'CAGR': 0.322343756160115,
 'IR': 3.1828474298797262}

In [ ]:
plot_signal(best_result, 5, 201)

In [ ]:
plot_returns(bh_results, best_result)